In [2]:
from labdata.schema import *
from labdata import chronic_paper

[2024-05-28 23:40:21,521][INFO]: Connecting admin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306
[2024-05-28 23:40:22,099][INFO]: Connected admin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306


In [36]:
# TODO: add missing procedures for mice
insertions_query = Procedure() & 'procedure_type = "chronic implant"'
chronic_mice = np.unique(insertions_query.fetch('subject_name'))
chronic_sessions_query = EphysRecording() & [dict(subject_name = c) for c in chronic_mice]
n_sorted_prbs = len(SpikeSorting() & 'parameter_set_num = 5')

chronic_probe_ids = np.unique((EphysRecording.ProbeSetting() & chronic_sessions_query).fetch('probe_id')).astype(int)

insertions = ((pd.DataFrame((EphysRecording.ProbeSetting() & chronic_sessions_query).proj('probe_id').fetch())[
               ['subject_name','probe_id']].drop_duplicates()))


reused = 0
for p in chronic_probe_ids:
    n = len((ProbeInsertion() & f'probe_id = "{p}"'))
    if n>1:
        reused += n-1 # dont count with the first use
reused_percent = int(100*reused/len(insertions))
print(f'We performed {len(insertions)} chronic insertions ({len(chronic_probe_ids)} probes) into {len(chronic_mice)} mice and recorded {len(chronic_sessions_query)} sessions.')
extract_percent = int(100*(len(((ProbeExtraction() & "extraction_successful = 1"))))/len(ProbeExtraction()))
print(f'We extracted {len((ProbeExtraction() & "extraction_successful = 1"))} functional probes successfully ({extract_percent}%).')
print(f'We re-implanted probes {reused} times ({reused_percent}%).')
print(f'We spike sorted  {n_sorted_prbs} probe*sessions.')
# insertions

We performed 42 chronic insertions (27 probes) into 16 mice and recorded 717 sessions.
We extracted 32 functional probes successfully (76%).
We re-implanted probes 16 times (38%).
We spike sorted  1381 probe*sessions.
